In [1]:
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.feature_selection import RFE
import pandas as pd

C:\Users\HARSHA\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
data = pd.read_csv('clustered_data.csv')
#print(data.count())
data = data[data.columns]
#print(data.head(5))

In [3]:
import numpy as np
list_data = data["Findings"]
y = np.array(data["Predictions"])
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

In [4]:
from collections import Counter
Counter(list_data)

Counter()
import re
Cleansed_data=[]
for j in list_data:
    Special_chars = re.sub(r'[\-\!\@\#\$\%\^\&\*\(\)\_\+\[\]\;\'\,\/\{\}\:\"\<\>\?\|]','',j)
    lower = Special_chars.lower()
    Widespace = lower.strip()
    Cleansed_data.append(Widespace)
#print(Cleansed_data[0:5])
print(len(Cleansed_data))

5688


In [5]:
vectorizer = CountVectorizer(stop_words='english')
X = vectorizer.fit(Cleansed_data)
#print(X)
vector = vectorizer.transform(Cleansed_data)
#print(vector)
print(vector.shape)
x = vector.toarray()
print(x[1].shape)
for i in range(len(x)):
    x[i] = x[i].reshape(-1, 1, 5352)
print(x[1].shape)
print(x)


(5688, 5352)
(5352,)
(5352,)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [10]:
# deepneuralnet.py
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from tflearn.metrics import Accuracy
acc = Accuracy()
network = input_data(shape=[None, 1,5352,1])
# Conv layers
network = conv_2d(network, 64, 3, strides=1, activation='relu', name = 'conv1_3_3_1')
network = max_pool_2d(network, 2, strides=2)
network = conv_2d(network, 64, 3, strides=1, activation='relu', name = 'conv1_3_3_2')
network = conv_2d(network, 64, 3, strides=1, activation='relu', name = 'conv1_3_3_3')
network = max_pool_2d(network, 2, strides=2)
# Fully Connected Layer 
network = fully_connected(network, 1024, activation='tanh')
# Dropout layer
network = dropout(network, 0.5)
# Fully Connected Layer
network = fully_connected(network, 1, activation='linear')
# Final network
network = regression(network, optimizer='momentum',
                     loss='categorical_crossentropy',
                     learning_rate=0.001, metric=acc)

# The model with details on where to save
# Will save in current directory
model = tflearn.DNN(network)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state = 42)

In [12]:

X_train = X_train.reshape([-1, 1, 1,5352])
X_test = X_test.reshape([-1, 1, 1, 5352])
y_train = np.reshape(y_train, (-1, 1))
y_test = np.reshape(y_test, (-1, 1))
model.fit(X_train, y_train, n_epoch=15, validation_set=(X_test, y_test), show_metric=True, run_id="deep_nn")
model.save('final-model.tflearn')

IndexError: list index out of range